In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten
from keras.layers import Dense
import numpy as np
import pylab as plt

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

In [2]:
# (n_frames, width, height, channels) and 返回形状相同的电影.
def mymodel():
    seq = Sequential()
    seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 24, 72, 1),
                   padding='same', return_sequences=True))
    #input_shape(None, 24, 72, 40)
    seq.add(BatchNormalization())
    seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
    seq.add(BatchNormalization())
    seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
    seq.add(BatchNormalization())

    seq.add(ConvLSTM2D(filters=30, kernel_size=(3, 3),
                   padding='same', return_sequences=False))
    seq.add(BatchNormalization())

    seq.add(Conv2D(filters=1, kernel_size=(3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
    seq.add(Flatten())
    seq.add(Dense(units=1024))
    seq.add(Dense(units=512))
    seq.add(Dense(units=24))
    seq.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
    return seq

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 24, 72, 40)  59200     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 24, 72, 40)  115360    
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 24, 72, 40)  115360    
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_4 (ConvLSTM2D)  (None, 24, 72, 30)       

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#创建三个模型 模型之间因为dropout的原因不尽相同
model1 = KerasClassifier(build_fn=mymodel, epochs=50, batch_size=15)
model2 = KerasClassifier(build_fn=mymodel, epochs=50, batch_size=15)
model3 = KerasClassifier(build_fn=mymodel, epochs=50, batch_size=15)

In [4]:
#data
import numpy as np
import netCDF4
from netCDF4 import Dataset
inp1 = Dataset('../data/enso_round1_train_20210201/CMIP_train.nc','r')
inp2 = Dataset('../data/enso_round1_train_20210201/CMIP_train.nc','r')

#[字序列数，子序列长度，长，宽]
inpv1 = np.zeros((4645,48,24,72))

#[4645, 48, 24, 72]
#通道压缩到序列中

inpv1[:,0:12,:,:] = inp1.variables['sst1'][:,0:12,:,:]
#c1 [2960, 第二年, :, :, ]
inpv1[:,12:24,:,:] = inp1.variables['t300'][:,0:12,:,:]
#c2 [2960, 第二年, :, :,]
inpv1[:,24:36,:,:] = inp1.variables['ua'][:,0:12,:,:]
inpv1[:,36:48,:,:] = inp1.variables['va'][:,0:12,:,:]

In [5]:
trX = inpv1[:,:,:,:]
trX.shape

(2960, 24, 24, 72)

In [6]:
trX=trX[:, : ,: ,: , np.newaxis]
trX.shape

(2960, 24, 24, 72, 1)

In [7]:
inpv2 = np.zeros((4645,24))
inpv2[:,0:24] = inp2.variables['nino'][:,12:36,0,0]

# [2961, 24]


trY = inpv2[:,:] 
trY.shape

(2960, 24)

In [ ]:
#训练


history = seq.fit(trX[:1000,:], trY[:1000,:], batch_size=16,
        epochs=100, validation_split=0.05)


Instructions for updating:
Use tf.cast instead.
Train on 950 samples, validate on 50 samples
Epoch 1/100


In [ ]:
ensemble_clf = VotingClassifier(estimators=[
    ('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')

ensemble_clf.fit(x_train, y_train)
y_pred = ensemble_clf.predict(x_test)
print('acc: ', accuracy_score(y_pred, y_test))

In [ ]:
#test data 
inp11 = Dataset('GODAS.input.36mon.1980_2015.nc','r')
inp22 = Dataset('GODAS.label.12mon.1982_2017.nc','r')

#[字序列数，子序列长度，长，宽]
inpv11 = np.zeros((35,24,24,72))

#[35, 24, 24, 72]
#通道压缩到序列中

inpv11[:,0:12,:,:] = inp11.variables['sst'][0:35,12:24,:,:]

inpv11[:,12:24,:,:] = inp11.variables['t300'][0:35,12:24,:,:]

testX = inpv11[:,:,:,:]
testX=testX[:, : ,: ,: , np.newaxis]
print(testX.shape)


inpv22 = np.zeros((35,24))
inpv22[:,0:12] = inp22.variables['pr'][0:35,:,0,0]
inpv22[:,12:24] = inp22.variables['pr'][1:36,:,0,0]

# [35, 24]


testY = inpv22[:,:] 
testY.shape

In [ ]:
loss,accuracy = seq.evaluate(x = testX,y = testY)
print(loss,accuracy)

In [ ]:
seq.save('./data/keras_model.h5')  

del model  #删除现有模型

# identical to the previous one
model = models.load_model('./data/keras_model.h5')
model.evaluate(x_test,y_test)

In [ ]:

import matplotlib.pyplot as plt

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.savefig('.\\data\\1.png')
    plt.show()

In [ ]:
plot_metric(history,"loss")

In [ ]:
plot_metric(history,"accuracy")